<a href="https://colab.research.google.com/github/valering97/Contacts-agenda/blob/main/ContactsAgenda_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Software for managing a contact agenda



In [ ]:
!mkdir rubrica

In [ ]:
%%writefile rubrica/__init__.py

Writing rubrica/__init__.py


In [ ]:
%%writefile main.py

import sys
import os
from rubrica.contact import Contact, ask_new_contact, read_from_json
from rubrica.manageMenu import print_menu, make_choice
from rubrica.manageContactsList import ContactList

def main(lista_contatti=None):
    if not lista_contatti:
        lista_contatti = ContactList()
    while True:
        print_menu()
        if not make_choice(lista_contatti):
            break
        continue_program = input("Vuoi fare altre operazioni? [s/n]: ")
        if continue_program.lower() != "s":
            print("Uscita dal programma, Arrivederci!")
            break

if __name__ == "__main__":
    main()


Writing main.py


In [ ]:
%%writefile rubrica/contact.py

import json
import os

FILE_PATH = 'contacts.json'

class Contact:
    def __init__(self, first_name, last_name, phone):
        self.first_name = first_name
        self.last_name = last_name
        self.phone = phone

    def __str__(self):
        return f"{self.first_name} {self.last_name}: {self.phone}"

    def update_contact(self, first_name=None, last_name=None, phone=None):
        if first_name:
            self.first_name = first_name
        if last_name:
            self.last_name = last_name
        if phone:
            self.phone = phone

    def matches(self, name):
        full_name = f"{self.first_name} {self.last_name}"
        return name.lower() in full_name.lower()

def ask_new_contact():
    first_name = input("Nome: ")
    while not first_name.isalpha():
        print("Attenzione, inserire un nome valido.")
        first_name = input("Nome: ")
    last_name = input("Cognome: ")
    while not last_name.isalpha():
        print("Attenzione, inserire un cognome valido.")
        last_name = input("Cognome: ")
    phone = input("Telefono: ")
    while not phone.isnumeric():
        print("Attenzione, inserire un numero di telefono valido.")
        phone = input("Telefono: ")
    return first_name, last_name, phone

def read_from_json():
    contacts = []
    if os.path.exists(FILE_PATH):
        try:
            with open(FILE_PATH, 'r') as contatti_json:
                data = json.load(contatti_json).get("Contatti", [])
                for d in data:
                    contacts.append(Contact(d['first_name'], d['last_name'], d['phone']))
        except json.JSONDecodeError:
            print("Errore nella lettura del file JSON.")
    return contacts


Overwriting rubrica/contact.py


In [ ]:
%%writefile rubrica/manageMenu.py
from rubrica.contact import Contact, ask_new_contact
import sys

def print_menu():
    print("--------------------------------")
    print("| RUBRICA CONTATTI ContactEase |")
    print("--------------------------------")
    print("\n1) Visualizza i contatti in rubrica")
    print("2) Inserisci un nuovo contatto")
    print("3) Cerca contatto per nome o cognome")
    print("4) Modifica un contatto")
    print("5) Elimina un contatto")
    print("6) Esci dal programma\n")

# Funzione per gestire l'aggiunta del contatto
def add_new_contact(lista_contatti):
    while True:
        first_name, last_name, phone = ask_new_contact()
        contact = Contact(first_name, last_name, phone)
        if lista_contatti.add_contact(contact):
            print("Contatto aggiunto correttamente.")
        else:
            print("Errore: Un contatto con questo nome e numero di telefono esiste già.")

        continue_add = input("Vuoi aggiungere altri contatti? [s/n]: ")
        if continue_add.lower() == "n":
            break

# Funzione per gestire la scelta delle opzioni del menu
def make_choice(lista_contatti):
    while True:
        choice = input("Seleziona un'opzione: ")
        while choice not in ["1", "2", "3", "4", "5", "6"]:
            print("Scelta non valida. Inserisci un numero compreso fra 1 e 6.")
            choice = input("Seleziona un'opzione: ")

        if choice == '1': #Visualizza i contatti
            if not lista_contatti.view_contacts():
                continue_add = input("Vuoi aggiungere un nuovo contatto? [s/n]: ")
                if continue_add.lower() == "s":
                    # Richiama la funzione di aggiunta contatto
                    add_new_contact(lista_contatti)
                return True
            else:
                return True

        elif choice == '2': #Aggiunta contatto
            add_new_contact(lista_contatti)
            return True

        elif choice == '3':
            name = input("Inserisci il nome o il cognome da cercare: ")
            results = lista_contatti.search_contact(name)
            if results:
                print("Contatti trovati: ")
                for contact in results:
                    print(contact)
            else:
                print("Contatto non trovato.")
            return True

        elif choice == '4':
            name = input("Inserisci il nome o il cognome del contatto da modificare: ")
            results = lista_contatti.search_contact(name)
            if len(results) == 1:
                print(results[0])
                contact = results[0]
                lista_contatti.edit_contact(contact)
            elif len(results) > 1:
                print("Contatti trovati: ")
                for i, contact in enumerate(results, start=1):
                    print(f"{i}) {contact}")
                try:
                    choice = int(input("Seleziona il contatto da modificare: "))
                    if choice in range(1, len(results) + 1):
                        contact = results[choice - 1]
                        lista_contatti.edit_contact(contact)
                    else:
                        print("Scelta non valida.")
                except ValueError:
                    print("Scelta non valida.")
            else:
                print("Contatto non trovato.")
            return True

        elif choice == '5':
            name = input("Inserisci il nome o il cognome del contatto da eliminare: ")
            results = lista_contatti.search_contact(name)
            if len(results) == 1:
                print(results[0])
                confirm = input("Sei sicuro di voler eliminare questo contatto? [s/n]: ")
                if confirm.lower() == "s":
                    lista_contatti.remove_contact(results[0])
                    print("Contatto eliminato correttamente.")
            elif len(results) > 1:
                print("Contatti trovati: ")
                for i, contact in enumerate(results, start=1):
                    print(f"{i}) {contact}")
                try:
                    choice = int(input("Seleziona il contatto da eliminare: "))
                    if choice in range(1, len(results) + 1):
                        contact = results[choice - 1]
                        confirm = input(f"Sei sicuro di voler eliminare {contact}? [s/n]: ")
                        if confirm.lower() == "s":
                            lista_contatti.remove_contact(contact)
                            print("Contatto eliminato correttamente.")
                        else:
                            print("Eliminazione annullata.")
                    else:
                        print("Scelta non valida.")
                except ValueError:
                    print("Scelta non valida.")
            else:
                print("Contatto non trovato.")
            return True

        elif choice == '6':
            exit_input = input("Sei sicuro di voler uscire dal programma? [s/n]: ")
            if exit_input.lower() == "s":
                print("Uscita dal programma, Arrivederci!")
                return False
            else:
                return True


Overwriting rubrica/manageMenu.py


In [ ]:
%%writefile rubrica/manageContactsList.py

import json
from rubrica.contact import Contact, read_from_json, FILE_PATH, ask_new_contact

class ContactList:
    """
    Classe per gestire la rubrica di contatti.

    Attributi:
        contacts (list): Lista dei contatti caricati dal file JSON.

    Metodi:
        __init__: Inizializza la rubrica caricando i contatti dal file JSON.
        is_duplicate: Verifica se un contatto è già presente nella rubrica.
        add_contact: Aggiunge un contatto alla rubrica, se non è duplicato.
        remove_contact: Rimuove un contatto dalla rubrica.
        view_contacts: Visualizza tutti i contatti nella rubrica.
        search_contact: Cerca contatti per nome o cognome.
        edit_contact: Modifica di un contatto esistente.
        write_to_json: Salva la lista dei contatti nel file JSON.
    """

    def __init__(self):
        self.contacts = read_from_json()

    def is_duplicate(self, contact):
        for c in self.contacts:
            if c.first_name == contact.first_name and c.last_name == contact.last_name and c.phone == contact.phone:
                return True
        return False

    def add_contact(self, contact):
        if self.is_duplicate(contact):
            return False
        self.contacts.append(contact)
        self.write_to_json()
        return True

    def remove_contact(self, contact):
        self.contacts.remove(contact)
        self.write_to_json()

    def view_contacts(self):
        if not self.contacts:
            print("Nessun contatto esistente.")
            return False
        else:
            print("Lista contatti in rubrica:")
            for contact in self.contacts:
                print(contact)
            return True

    def search_contact(self, name):
        results = []
        for contact in self.contacts:
            if contact.matches(name):
                results.append(contact)
        return results

    def edit_contact(self, contact):
        first_name, last_name, phone = ask_new_contact()
        contact.update_contact(first_name, last_name, phone)
        self.write_to_json()
        print("Contatto modificato correttamente.")

    def write_to_json(self):
        data = {'Contatti': [{'first_name': c.first_name, 'last_name': c.last_name, 'phone': c.phone} for c in self.contacts]}
        with open(FILE_PATH, 'w') as contatti_json:
            json.dump(data, contatti_json, indent=4)


Overwriting rubrica/manageContactsList.py


In [ ]:
!python3 main.py

--------------------------------
| RUBRICA CONTATTI ContactEase |
--------------------------------

1) Visualizza i contatti in rubrica
2) Inserisci un nuovo contatto
3) Cerca contatto per nome o cognome
4) Modifica un contatto
5) Elimina un contatto
6) Esci dal programma

Seleziona un'opzione: 1
Lista contatti in rubrica:
vale vale: 1
Vuoi fare altre operazioni? [s/n]: s
--------------------------------
| RUBRICA CONTATTI ContactEase |
--------------------------------

1) Visualizza i contatti in rubrica
2) Inserisci un nuovo contatto
3) Cerca contatto per nome o cognome
4) Modifica un contatto
5) Elimina un contatto
6) Esci dal programma

Seleziona un'opzione: 2
Nome: valeria
Cognome: rossi
Telefono: 331429
Contatto aggiunto correttamente.
Vuoi aggiungere altri contatti? [s/n]: s
Nome: lorenzo
Cognome: rossini
Telefono: 339429
Contatto aggiunto correttamente.
Vuoi aggiungere altri contatti? [s/n]: n
Vuoi fare altre operazioni? [s/n]: s
--------------------------------
| RUBRICA CONTA